# 读取mat文件

In [4]:
import scipy.io as sio
import numpy as np
from collections import defaultdict
import gurobipy as gp
import time

In [5]:
def read_mat(filepath:str) :
    """读取.mat网络文件
    用python读取mat文件时，所有数据元素都会变成二维数组

    Args:
        filepath (str): 文件完整路径

    Returns:
        node_num,nodes_dict,edges_dict,neighbor_nodes_dict: 节点属性字典,边属性字典,邻居节点字典
    """
    
    
    mat_data = sio.loadmat(file_name=filepath)
    node_data = mat_data['PlanarNetwork']['Node'][0,0]
    edge_data = mat_data['PlanarNetwork']['Edge'][0,0]
    
    # 保存node节点中数据,{节点:(经度,维度)}
    nodes_dict = {}
    node_num = node_data.shape[1]
    for i in range(node_num):
        nodeID = node_data[0,i][0][0,0]-1
        longitude = node_data[0,i][1][0,0]
        latitude = node_data[0,i][2][0,0]
        nodes_dict[nodeID] = (longitude,latitude)
    
    # 保存edge节点中的数据,{(起点,终点):长度},以及邻居节点{节点:{邻居节点}}
    edges_dict = {}
    neighbor_nodes_dict = defaultdict(set)
    edge_num = edge_data.shape[1]
    for i in range(edge_num):
        fromNodeID = edge_data[0,i][1][0,0]-1
        endNodeID = edge_data[0,i][2][0,0]-1
        edge_len = edge_data[0,i][3][0,0]
        edges_dict[(fromNodeID,endNodeID)] = edge_len
        neighbor_nodes_dict[fromNodeID].add(endNodeID)
    
    return nodes_dict,edges_dict,neighbor_nodes_dict
    

In [11]:
# 模型所需参数
nodes_dict,edge_dict,neighbor_nodes_dict = read_mat('./RandomPlanarNetworks_200/PlanarNetwork_N200_E550_S100.mat')
c = 275       # 删除边的数目

# 方法一：整数线性规划模型
v_ij表示边ij是否被剔除
u_ij表示节点i->j是否连通

In [28]:
# 构建模型
node_num = len(nodes_dict)
edge_num = len(edge_dict)
model = gp.Model()

In [29]:
# 添加决策变量
v = {}
for (i,j) in edge_dict.keys():
    v[(i,j)] = model.addVar(vtype=gp.GRB.BINARY,name='v_{}_{}'.format(i,j))
u = []
for i in range(node_num):
    u_i = []
    for j in range(node_num):
        u_ij = model.addVar(vtype=gp.GRB.BINARY,name='u_{}_{}'.format(i,j))
        u_i.append(u_ij)
    u.append(u_i)
L = model.addVar(lb=0,ub=node_num,vtype=gp.GRB.INTEGER,name='L')        # 最大连通数
model.update()


In [30]:
# 添加约束
# 约束1：删除的边小于等于特定数目
model.addConstr(gp.quicksum(list(v.values()))<=c)
# 约束2：邻居节点连通性约束下界
for (i,j) in edge_dict.keys():
    model.addConstr(u[i][j] >= 1-v[(i,j)])
# 约束3：邻居节点连通性约束上界
for (i,j) in edge_dict.keys():
    model.addConstr(u[i][j] <= gp.quicksum([u[k][j] for k in neighbor_nodes_dict[i]]) + v[(i,j)])
# 约束4：非邻居节点连通性约束下界
for i in range(node_num-1):
    for j in range(i+1,node_num):
        for k in neighbor_nodes_dict[i]:
            if k == j:
                continue
            model.addConstr(u[i][j] >= u[k][j]-v[(i,k)])
# 约束5：非邻居节点连通性约束上界
for i in range(node_num-1):
    for j in range(i+1,node_num):
        if (i,j) not in edge_dict:
            model.addConstr(u[i][j] <= gp.quicksum([u[k][j] for k in neighbor_nodes_dict[i] if k != j]))
# 约束6：连通性的对称性
for i in range(node_num):
    for j in range(node_num):
        model.addConstr(u[i][j] == u[j][i])
# 约束7：最大连通子图的节点数
for i in range(node_num):
    model.addConstr(L>=gp.quicksum(u[i]))
# 约束8：节点自己的连通性为1
for i in range(node_num):
    model.addConstr(u[i][i] == 1)

In [31]:
# 添加目标函数
model.setObjective(L,sense=gp.GRB.MINIMIZE)

In [32]:
model.Params.LogToConsole=True # 显示求解过程
model.optimize()

Set parameter LogToConsole to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19043.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-11400H @ 2.70GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 133062 rows, 40551 columns and 431494 nonzeros
Model fingerprint: 0x2ba24256
Variable types: 0 continuous, 40551 integer (40550 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 5e+00]
Presolve removed 119948 rows and 36033 columns
Presolve time: 0.29s
Presolved: 13114 rows, 4518 columns, 47875 nonzeros
Variable types: 0 continuous, 4518 integer (4517 binary)
Found heuristic solution: objective 109.0000000
Found heuristic solution: objective 108.0000000
Performing another presolve...
Presolve removed 47 rows and 42 columns
Presolve time: 0.11s
Found heuristic sol

In [26]:
# 查看决策变量取值
for var in model.getVars():
   print(f"{var.varName}: {var.X}")

v_0_184: -0.0
v_0_188: -0.0
v_1_105: 0.0
v_1_193: -0.0
v_2_18: -0.0
v_2_137: -0.0
v_3_98: -0.0
v_3_143: -0.0
v_4_17: -0.0
v_4_26: 1.0
v_5_58: 0.0
v_5_83: -0.0
v_5_150: -0.0
v_6_25: 1.0
v_6_64: -0.0
v_7_80: -0.0
v_7_97: -0.0
v_7_152: -0.0
v_8_148: -0.0
v_8_153: 0.0
v_8_197: -0.0
v_9_30: 0.0
v_9_35: -0.0
v_10_81: -0.0
v_10_91: -0.0
v_11_39: 0.0
v_11_74: -0.0
v_12_18: -0.0
v_12_159: -0.0
v_13_41: -0.0
v_13_75: 0.0
v_13_92: -0.0
v_14_41: -0.0
v_14_198: -0.0
v_15_45: -0.0
v_15_49: 0.0
v_15_125: -0.0
v_16_174: 0.0
v_17_167: -0.0
v_17_177: -0.0
v_17_183: -0.0
v_18_71: -0.0
v_19_29: -0.0
v_19_42: -0.0
v_19_145: -0.0
v_20_46: -0.0
v_20_114: -0.0
v_21_39: 0.0
v_21_72: -0.0
v_21_85: -0.0
v_22_106: -0.0
v_22_144: 0.0
v_22_159: -0.0
v_23_33: -0.0
v_23_77: -0.0
v_23_194: 0.0
v_24_32: -0.0
v_24_96: -0.0
v_25_135: -0.0
v_25_140: -0.0
v_25_149: -0.0
v_26_63: 0.0
v_26_118: -0.0
v_26_145: -0.0
v_27_30: 0.0
v_27_93: 0.0
v_28_158: -0.0
v_29_85: -0.0
v_29_165: -0.0
v_31_166: 0.0
v_32_158: -0.0
v_32_187: -0.

## Gurobi封装类

In [13]:
class Model:
    def __init__(self,nodes_dict,edge_dict,neighbor_nodes_dict,c):
        """

        :param nodes_dict: 节点属性字典
        :param edge_dict: 边属性字典
        :param neighbor_nodes_dict: 邻居节点字典
        :param c: 最大删除边数
        """
        self.nodes_dict = nodes_dict
        self.edge_dict = edge_dict
        self.neighbor_nodes_dict = neighbor_nodes_dict
        self.c = c
        self.node_num = len(nodes_dict)
        self.edge_num = len(edge_dict)
        self.model = None
        self.solve_time = 0
        self.objVal = 0
        self.ObjBound = 0

    def build_model(self):
        """构建Gurobi模型"""
        model = gp.Model()
        # 添加决策变量
        v = {}
        for (i,j) in self.edge_dict.keys():
            v[(i,j)] = model.addVar(vtype=gp.GRB.BINARY,name='v_{}_{}'.format(i,j))
        u = []
        for i in range(self.node_num):
            u_i = []
            for j in range(self.node_num):
                u_ij = model.addVar(vtype=gp.GRB.BINARY,name='u_{}_{}'.format(i,j))
                u_i.append(u_ij)
            u.append(u_i)
        l = model.addVar(lb=0,ub=self.node_num,vtype=gp.GRB.INTEGER,name='L')        # 最大连通数
        model.update()

        # 添加目标函数
        model.setObjective(l,sense=gp.GRB.MINIMIZE)

        # 添加约束
        # 约束1：删除的边小于等于特定数目
        model.addConstr(gp.quicksum(list(v.values()))<=c)
        # 约束2：邻居节点连通性约束下界
        for (i,j) in self.edge_dict.keys():
            model.addConstr(u[i][j] >= 1-v[(i,j)])
        # 约束3：邻居节点连通性约束上界
        for (i,j) in self.edge_dict.keys():
            model.addConstr(u[i][j] <= gp.quicksum([u[k][j] for k in self.neighbor_nodes_dict[i]]) + v[(i,j)])
        # 约束4：非邻居节点连通性约束下界
        for i in range(self.node_num-1):
            for j in range(i+1,self.node_num):
                for k in self.neighbor_nodes_dict[i]:
                    if k == j:
                        continue
                    model.addConstr(u[i][j] >= u[k][j]-v[(i,k)])
        # 约束5：非邻居节点连通性约束上界
        for i in range(self.node_num-1):
            for j in range(i+1,self.node_num):
                if (i,j) not in edge_dict:
                    model.addConstr(u[i][j] <= gp.quicksum([u[k][j] for k in self.neighbor_nodes_dict[i] if k != j]))
        # 约束6：连通性的对称性
        for i in range(self.node_num):
            for j in range(self.node_num):
                model.addConstr(u[i][j] == u[j][i])
        # 约束7：最大连通子图的节点数
        for i in range(self.node_num):
            model.addConstr(l>=gp.quicksum(u[i]))
        # 约束8：节点自己的连通性为1
        for i in range(self.node_num):
            model.addConstr(u[i][i] == 1)

        self.model = model

    def solve(self,TimeLimit=None,):
        """
        模型求解
        """
        self.model.resetParams()     # 清除超参数设置
        if TimeLimit is not None:
            self.model.Params.TimeLimit = TimeLimit
        self.model.Params.LogToConsole=True # 显示求解过程
        t = time.time()
        self.model.optimize()
        self.solve_time = time.time()-t
        self.objVal = self.model.objVal
        self.ObjBound = self.model.ObjBound

    def get_vars(self):
        """查看决策变量取值"""
        for var in self.model.getVars():
            print(f"{var.varName}: {var.X}")

In [14]:
model = Model(nodes_dict,edge_dict,neighbor_nodes_dict,c)
model.build_model()
model.solve()
print('最优值：{}；最优下界：{}；求解时间：{}'.format(model.objVal,model.ObjBound,model.solve_time))

Reset all parameters
Set parameter LogToConsole to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19043.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-11400H @ 2.70GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 133062 rows, 40551 columns and 431494 nonzeros
Model fingerprint: 0x5739da39
Variable types: 0 continuous, 40551 integer (40550 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 3e+02]
Presolve removed 119948 rows and 36033 columns
Presolve time: 0.25s
Presolved: 13114 rows, 4518 columns, 47875 nonzeros
Variable types: 0 continuous, 4518 integer (4517 binary)
Found heuristic solution: objective 53.0000000
Found heuristic solution: objective 52.0000000
Performing another presolve...
Presolve removed 47 rows and 42 columns
Presolve time: 0.08s


# 方法二：